In [1]:
import asyncio 
import matplotlib

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient, resample

%matplotlib inline

In [3]:
efd_client = EfdClient('summit_efd')

Query for all the `endReadout` events on the timespan of the night.

In [5]:
t1 = Time("2020-01-23T00:21:00", format='isot', scale='tai')
t2 = Time("2020-01-23T06:59:00", format='isot', scale='tai')

In [16]:
end_readout = await efd_client.select_time_series("lsst.sal.GenericCamera.logevent_endReadout", 
                                           ["imageName", "exposureTime"], t1, t2)

Now match each entry. For each `i` item with `intra` in the name, there must be an `i+1` with `extra` otherwise it is not a pair. 

In [17]:
intra_images = []
extra_images = []
intra_times = []
extra_times = []
intra_exptimes = []
extra_exptimes = []

i = 0
npairs = 0
nmiss = 0

while i < len(end_readout):
    intra = end_readout['imageName'][i]
    extra = end_readout['imageName'][i+1]
    
    if 'intra' in intra and 'extra' in extra:
#         print(f"Got a pair: {intra} x {extra}")
        intra_images.append(intra)
        extra_images.append(extra)
        intra_times.append(end_readout.index[i])
        extra_times.append(end_readout.index[i+1])
        intra_exptimes.append(end_readout['exposureTime'][i])
        extra_exptimes.append(end_readout['exposureTime'][i+1])
        i+=2
        npairs+=1
    else:
#         print(f"No Match: {intra} x {extra}")
        nmiss+=1
        i+=1

print(f"Got {npairs} pairs and {nmiss} misses.")

Got 68 pairs and 2 misses.


In [12]:
match_data = np.zeros(npairs, dtype=[('intra', '<U35'), 
                                           ('extra', '<U35'), 
                                           ('az', float), 
                                           ('el', float), 
                                           ('rot_pos', float),
                                           ('x', float), 
                                           ('y', float), 
                                           ('z', float), 
                                           ('u', float), 
                                           ('v', float), 
                                           ('w', float)])

In [42]:
for i in range(npairs):

    
    t1 = Time(intra_times[i], scale='tai') - TimeDelta(intra_exptimes[i], format='sec', scale='tai') - TimeDelta(30., format='sec', scale='tai')
    t2 = Time(extra_times[i], scale='tai') - TimeDelta(1., format='sec', scale='tai')

    azel = await efd_client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", 
                                               ["elevationCalculatedAngle99", "azimuthCalculatedAngle99"], t1, t2)
    rotator = await efd_client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders",
                                                  ["nasmyth1CalculatedAngle99"], t1, t2)
    hexapod = await efd_client.select_time_series("lsst.sal.ATHexapod.command_moveToPosition",
                                                  ["x", "y", "z", "u", "v", "w"], t1, t2)
    if len(hexapod) < 2:
        print(f"Could not get hexapod position for pair {i+1}: {intra_images[i]} x {extra_images[i]}")
        continue
    elif '1579755351' in intra_images[i]:
        print(f"{intra_times[i]}:{intra_images[i]}")
        print(hexapod)
        print(f"{extra_times[i]}:{extra_images[i]}")

    rot_pos = np.mean(rotator['nasmyth1CalculatedAngle99'])
    el = np.mean(azel['elevationCalculatedAngle99'])
    az = np.mean(azel['azimuthCalculatedAngle99'])
    x = (hexapod['x'][len(hexapod)-2]+hexapod['x'][len(hexapod)-1])/2.
    y = (hexapod['y'][len(hexapod)-2]+hexapod['y'][len(hexapod)-1])/2.
    z = (hexapod['z'][len(hexapod)-2]+hexapod['z'][len(hexapod)-1])/2.
    u = (hexapod['u'][len(hexapod)-2]+hexapod['u'][len(hexapod)-1])/2.
    v = (hexapod['v'][len(hexapod)-2]+hexapod['v'][len(hexapod)-1])/2.
    w = (hexapod['w'][len(hexapod)-2]+hexapod['w'][len(hexapod)-1])/2.
    match_data[i] = (intra_images[i], extra_images[i], az, el, rot_pos, x, y, z, u, u, w)
#     print(az, el, rot_pos, x, y, z, u, u, w)
print("done")

2020-01-23 04:56:22.415735385+00:00:1579755351-HD72660_intra-0-1
                                            x         y         z         u  \
2020-01-23 04:55:41.537691489+00:00 -4.449999  0.099999  0.470002  0.350001   
2020-01-23 04:55:54.077718486+00:00 -4.449999  0.099999 -1.029998  0.350001   
2020-01-23 04:56:22.529461649+00:00 -4.449999  0.099999  1.970002  0.350001   

                                        v  w  
2020-01-23 04:55:41.537691489+00:00  0.22  0  
2020-01-23 04:55:54.077718486+00:00  0.22  0  
2020-01-23 04:56:22.529461649+00:00  0.22  0  
2020-01-23 04:57:15.423734808+00:00:1579755389-HD72660_extra-0-1
done


In [44]:
df = pd.DataFrame(match_data)

In [45]:
df.to_csv("20200122_match.csv")

In [36]:
np.savetxt("20200122_matched_data.txt", np.array(intra_images +extra_images), fmt="%s")

In [39]:
(1.970002+0.470002)/2.

1.220002

In [43]:
(-1.029998+1.970002)/2.

0.47000200000000003